In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def dailyChart():
    # TradingView 미국 활발한 주식 페이지 URL
    url_volume = "https://kr.tradingview.com/markets/stocks-usa/market-movers-active/"
    url_change = "https://kr.tradingview.com/markets/stocks-usa/market-movers-gainers/"

    # HTTP 요청
    response_volume = requests.get(url_volume)
    response_volume.raise_for_status()  # 요청 성공 여부 확인

    response_change = requests.get(url_change)
    response_change.raise_for_status()  # 요청 성공 여부 확인

    # BeautifulSoup으로 HTML 파싱
    soup_volume = BeautifulSoup(response_volume.text, 'html.parser')
    soup_change = BeautifulSoup(response_change.text, 'html.parser')

    # 필요한 데이터 추출 (예: 티커, 회사명, 가격, 변동률 등)
    ticker_volume = []
    ticker_change = []
    volume = []
    changePer = []

    # 테이블 행을 순회하며 정보 추출
    for row in soup_volume.select('table tr'):
        cols = row.find_all('td')
        if len(cols) > 4:  # 데이터가 있는 행인지 확인
            ticker_volume.append(cols[0].text.strip())       # 회사명
            volume.append(cols[1].text.strip())       # 거래량
    
    for row in soup_change.select('table tr'):
        cols = row.find_all('td')
        if len(cols) > 4:  # 데이터가 있는 행인지 확인
            ticker_change.append(cols[0].text.strip())       # 회사명
            changePer.append(cols[1].text.strip())    # 변화율

    # 데이터프레임 생성
    data_volume = pd.DataFrame({
        'Company': ticker_volume,
        'Volume': volume
    })

    data_change = pd.DataFrame({
        'Company': ticker_change,
        'changePercents': changePer
    })



    return data_volume,data_change



data_volume, data_change=dailyChart()

print(data_volume)
print(data_change)

                      Company        Volume
0             TSLATesla, Inc.  47.673 B USD
1      NVDANVIDIA Corporation  35.244 B USD
2        AMZNAmazon.com, Inc.  14.971 B USD
3              AAPLApple Inc.  12.152 B USD
4   MSFTMicrosoft Corporation  11.211 B USD
..                        ...           ...
95       SLBSchlumberger N.V.   1.111 B USD
96       DHID.R. Horton, Inc.   1.106 B USD
97      TGTTarget Corporation   1.106 B USD
98          KKRKKR & Co. Inc.   1.098 B USD
99     MELIMercadoLibre, Inc.   1.077 B USD

[100 rows x 2 columns]
                              Company changePercents
0         CTKBCytek Biosciences, Inc.        +36.03%
1   TMCITreace Medical Concepts, Inc.        +35.07%
2           COINCoinbase Global, Inc.        +31.11%
3              CIFRCipher Mining Inc.        +31.02%
4                  TEMTempus AI, Inc.        +28.02%
..                                ...            ...
95     WSFSWSFS Financial Corporation        +16.20%
96            ARCBArcBes

In [ ]:
import requests
from bs4 import BeautifulSoup

# 네이버 환율 페이지 URL
url = "https://finance.naver.com/marketindex/"

# HTTP 요청
response = requests.get(url)
response.raise_for_status()

# BeautifulSoup으로 HTML 파싱
soup = BeautifulSoup(response.text, 'html.parser')

# 하나은행 기준 환율 데이터 추출 (예: 달러-원 환율)
usd_krw = soup.select_one('div.market1 div.head_info > span.value').text
print("USD to KRW:", usd_krw)


USD to KRW: 1,396.30


In [ ]:
import gradio as gr
from gradio_modal import Modal

def info_fn():
    information="""< 분석 기준 ><br>
    - open(시작가)<br>
    - close(종가)<br>
    - InterestRate(금리)<br>
    - ExchangeRate(1유로 기준 달러 환율)<br>
    - VIX(변동성 지수(=공포 지수))<br>
    - TEDSpread(은행 간 대출 금리와 단기 미국 국채 금리 간의 차이)<br>
    - EFFR(예금 기관들이 서로에게 하룻밤 동안 예비 잔고를 대출할때 적용되는 금리)<br>
    - Gold(금 가격)<br>
    - Oil(유가)<br>
    - Close_InterestRate_Corr(종가와 이자율의 상관계수)<br>
    - Close_VIX_Corr(종가-공포지수 상관계수)<br>
    - Daily_Return(일일 수익률)<br>
    - Volatility(30일 이동평균편차)<br>
    - Rolling_Mean_Close(30일 이동평균)<br><br>

    < 분석 기준 출처 ><br>
    - open,close,Gold, Oil = YFinance<br>
    - InterestRate, ExchangeRate, VIX,<br> TEDSpread, EFFR = FRED<br>
    - 달러 환율=네이버 은행고시환율(하나은행)<br><br>

    < 분석 기간 ><br>
    2010-01-01 ~ 2024-11-06<br><br>

    < 분석 모델 ><br> 
    RandomForestRegression<br><br>
    """
    gr.Info(information,duration=None,title="")

with gr.Blocks() as demo:
    with gr.Row():
        trigger_info = gr.Button(value="AI 관련 정보")

        trigger_info.click(info_fn, None, None)

if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7930

To create a public link, set `share=True` in `launch()`.
